# To-do
- Make a table showing results of previous day, export to excel sheet 
- Results can be amount of water from free session
- Amount of water from actual session
- Current SM and how many trials in each session for each SM
- Draw plots for all this, keep updating in the excel sheet

In [100]:
from opconNosepokeFunctions import *
import sys
sys.stdout(open("C:/Users/Rishika/Desktop/lab/ATM_analysis/pylog.out"), "w")
toneStartMarker = 23
trialStartMarker = 81
trialHitMarker = 51
trialMissMarker = 86
trialUnrewMarker = 87
sessionStartMarker = 61
rewardProbMarker = 83
stateMachines = [13]

arms = 4
current_animals = ['test05022023', 'Blissey', 'Chikorita', 'Darkrai', 'Eevee', 'Fennekin']
boxes = listdir_fullpath("L:/4portProb/")

# arms = 8
# current_animals = ['Blissey', 'Darkrai']
# boxes = listdir_fullpath("L:/8portProb/")

files = {}
df_dict = {}
sess_dict = {}

# get animal name from file
filenameFormat = ['name', 'yyyy', 'mm', 'dd', 'hh', 'nn', 'ss', 'dat']
split_name=[]
animal_name = []
date = []

for box in boxes:
    for animal in os.listdir(box): 
        if animal in current_animals:
            try:
                print()
                files[box] = exclude_files(get_files(box+'/'+os.listdir(box)[0]+'/'))[-1]
                print(files[box])
                split_name = files[box].replace('.', '-').split('-')
                animal_name.append(split_name[0])
                date.append('-'.join(split_name[-7:-4][::-1]))
                
                df_dict[box] = merge_files_to_df([files[box]])

            except pd.errors.EmptyDataError as e:
                print(animal)   
print('Animal names = ', animal_name,'\n','Last file datestamp = ', date)


test05022023-2023-04-24-17-07-32.dat

Blissey-2023-04-24-17-05-01.dat

Chikorita-2023-04-24-18-22-41.dat

Darkrai-2023-04-24-14-44-11.dat

Eevee-2023-04-25-08-45-25.dat

Fennekin-2023-04-25-08-51-03.dat
Animal names =  ['test05022023', 'Blissey', 'Chikorita', 'Darkrai', 'Eevee', 'Fennekin'] 
 Last file datestamp =  ['24-04-2023', '24-04-2023', '24-04-2023', '24-04-2023', '25-04-2023', '25-04-2023']


In [101]:
# get statemachine from SMtp ####################### PENDING #################################

In [103]:
# check how many boxes are being synced to NAS
boxes = listdir_fullpath('L:/4portProb/')

# write to excel!!
set_cwd("C:/Users/Rishika/Desktop/lab/ATM_analysis")

# loading workbook
wb = xl.load_workbook("trial.xlsx")
for box in boxes:
    if box =='L:/#recycle':
        pass
    else:
    # creating sheets for each box
        name = box.split('/')[-1]
        if name in wb.sheetnames:
            print(f'sheets okay, {name}')
        else:
            print(f'adding a new sheet for new box, {name}')
            wb.create_sheet(box.split('/')[-1], -1)

        # setting active sheet
        wb.active = wb[name]
        
        # to get results:
        # convert to df, using only last file here!
        df = df_dict[box]

        # convert .dat file to trial-based format
        data, n_trials, rew_trials, miss_trials = df_to_sm(df, stateMachines, trialStartMarker,
                                                           trialHitMarker, trialMissMarker, sessionStartMarker)
        # retreive session-wise data
        sessions = data['session#'].unique().astype(int)
        arr = [0,1,2]
        n_sess1, n_sess2, n_sess3 = 0,0,0
        rew_sess1, rew_sess2, rew_sess3 = 0,0,0
        miss_sess1, miss_sess2, miss_sess3 = 0,0,0
        
        if (len(sessions)!=3):
            missing_sess = [item for item in range(arr[0], arr[-1]+1) if item not in arr]
            print(missing_sess)             
        else: 
            print('all sess performed')
            n_sess1 = len(data[data['session#']==sessions[0]])
            rew_sess1 = data[(data['reward']==1) & (data['session#']==sessions[0])].shape[0]
            miss_sess1 = data[(data['reward']==0) & (data['session#']==sessions[0])].shape[0]
            n_sess2 = len(data[data['session#']==sessions[1]])
            rew_sess2 = data[(data['reward']==1) & (data['session#']==sessions[1])].shape[0]
            miss_sess2 = data[(data['reward']==0) & (data['session#']==sessions[1])].shape[0]
            n_sess3 = len(data[data['session#']==sessions[2]])
            rew_sess3 = data[(data['reward']==1) & (data['session#']==sessions[2])].shape[0]
            miss_sess3 = data[(data['reward']==0) & (data['session#']==sessions[2])].shape[0]   
        
        # write results to sheets
        ws = wb.active
        
        # freewater calc
        sMs = np.sort(df[2].unique())
        for j in sMs:
            if j==0:
                freewater = max(df[df[2]==0][6]) - df[df[2]==0][6].iat[0]
            elif j==1:
                freewater = max(df[df[2]==1][6]) - df[df[2]==1][6].iat[0]
        
        # ws format name, animal, date, SM, free water, etc.
        try:
            if (ws.cell(row=ws.max_row, column=3).value)=='date':
#                 print([name, animal_name, date, 'codepending', n_trials, rew_trials, miss_trials, freewater,
#                        n_sess1, rew_sess1, miss_sess1, n_sess2, rew_sess2, miss_sess2, n_sess3, rew_sess3, miss_sess3])
                ws.append([name, animal_name[boxes.index(box)], date[boxes.index(box)], 'code pending', 
                           n_trials, rew_trials, miss_trials,
                           freewater, n_sess1, rew_sess1, miss_sess1, n_sess2, rew_sess2,
                           miss_sess2, n_sess3, rew_sess3, miss_sess3])
                print('fish')

            elif (ws.cell(row=ws.max_row, column=3).value)!=date[boxes.index(box)]:
#                 print([name, animal_name, date, 'codepending', n_trials, rew_trials, miss_trials, freewater,
#                        n_sess1, rew_sess1, miss_sess1, n_sess2, rew_sess2, miss_sess2, n_sess3, rew_sess3, miss_sess3])
                ws.append([name, animal_name[boxes.index(box)], date[boxes.index(box)], 'code pending', n_trials, rew_trials, miss_trials, 'code pending',
                       n_sess1, rew_sess1, miss_sess1, n_sess2, rew_sess2, miss_sess2, n_sess3, rew_sess3, miss_sess3])
                print('fosh')
        
        except NameError as e:    
            print('Didnt write anything here.')
            pass
        
        print('done ', box)
        print('=====================================================')
    
# saving and done! - overwrites existing files!!
wb.save("trial.xlsx")
wb.close()
sys.stdout.close()

sheets okay, Box1
 no of trials in SMs =  (22,) 
 no of rewards in SMs =  (0,) 
 no of misses in SMs =  (22,) 
 rewards + misses =  22 

[]
done  L:/4portProb/Box1
sheets okay, Box2
 no of trials in SMs =  (1351,) 
 no of rewards in SMs =  (827,) 
 no of misses in SMs =  (524,) 
 rewards + misses =  1351 

all sess performed
done  L:/4portProb/Box2
sheets okay, Box3
 no of trials in SMs =  (2647,) 
 no of rewards in SMs =  (378,) 
 no of misses in SMs =  (630,) 
 rewards + misses =  1008 

all sess performed
done  L:/4portProb/Box3
sheets okay, Box4
 no of trials in SMs =  (161,) 
 no of rewards in SMs =  (56,) 
 no of misses in SMs =  (105,) 
 rewards + misses =  161 

all sess performed
done  L:/4portProb/Box4
sheets okay, Box5
 no of trials in SMs =  (0,) 
 no of rewards in SMs =  (0,) 
 no of misses in SMs =  (0,) 
 rewards + misses =  0 

[]
done  L:/4portProb/Box5
sheets okay, Box6
 no of trials in SMs =  (0,) 
 no of rewards in SMs =  (0,) 
 no of misses in SMs =  (0,) 
 rewards